In [111]:
%load_ext autoreload
import ast

from event_loop.activity_boundaries import ActivityBoundariesClassifier
from event_loop.activity_type import ActivityTypeClassifier
from event_loop.event_activity import EventActivityAssignment
from event_loop.preprocessing.dataframe import *
from sklearn.metrics import classification_report
import pandas as pd




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
df_train_in = pd.read_csv('../data_v3/hr_extended_features.csv', converters={"MessageAttributes": ast.literal_eval})
# This is the Interleaved Data Set for our pipeline
df_il_in = pd.read_csv('../data/HR-INTERLEAVED/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

# Load start and end events from ground truth data.
# Tag according frames in interleaved data for testing
df_gt = pd.read_csv("../data_v3/hr_ground_truth.csv")

start_indices = df_gt["start"].tolist()
end_indices = df_gt["actual_end"].tolist()

df_train = pre_process(df_train_in)
df_test = pre_process(df_il_in)

df_train = assign_sequence_number(df_train)

df_test["ActivityAction"] = df_test["frame.number"].apply(lambda x: "Activity Start" if x in start_indices else
("Activity End" if x in end_indices else "NoAction"))

df_train = mark_start_end(df_train)

In [13]:
%autoreload 2
activity_boundaries_classifier = ActivityBoundariesClassifier(df_train, None)



# Event Loop

In [14]:
records = df_il_in.to_dict("records")

In [60]:
from event_loop.event_activity import *
%autoreload 2

from event_loop.preprocessing.event import keep_event
import time
from event_loop.stack import *
from event_loop.event import Event


EVENT_LOOP_CUTOFF_NO_ACTION = 3
EVENT_LOOP_CUTOFF_END_EVENT = 3
ENTROPY_THRESHOLD = 0.4 #0.5
MAX_WINDOW_SIZE = 10
VERBOSE = False
SETTING = "HR"

# init variables
event_buffer: list[Event] = []
attribute_buffer: list[dict] = []
stacks: list[Stack] = []
stacks_out: list[Stack] = []
event_loop_index = 0


# TODO Change config to something like this:
config_dict = {
    "PTP": {
        "1to1" : ["applicant_id", "activity_id"],
        "1toN": ["mail_id"]
    }
}

HR_ATTRIBUTES = ["applicant_id", "activity_id"]
PTP_ATTRIBUTES = ["sale_order_id", "sale_order_line_id","purchase_requisition_id","purchase_requisition_line_id",]


#activity_model_data = get_activity_model_data(MAX_WINDOW_SIZE)


processing_times = []
buffer_sizes = []

for i, event_data in enumerate(records):
    buffer_sizes.append(sum([len(stack) for stack in stacks]))
    start_time = time.time()
    
    # Filter Event Stream
    if not keep_event(event_data):
        # skip event in loop
        continue
        
    # count every not filtered event for event loop index
    event_loop_index += 1
    
    # Extract Features and generate Event Object
    event = Event(event_data, event_loop_index, event_buffer, SETTING)
    event_buffer.append(event)
    
    activity_boundaries_classifier.classify_event(event) 
    activity_action = event.activity_action
    
    
        # Activity Matching
    if activity_action == "Activity Start": 
        stacks.append(Stack(SETTING,event))
        
    if activity_action == "NoAction": 
        if len(stacks) == 1: 
            stacks[0].append_event(event)
        elif event.origin_request_frame: 
            idx = search_stack_for_request_frame(event.origin_request_frame, stacks)
            stacks[idx].append_event(event)
        else: 
            # Check attributes of each stack
            
            # we can filter out stacks that already have attributes different to the event
            exclude_indices =  event_activity_model.exclude_stacks_by_attribute(stacks, event)
    
            stack_index:int = event_activity_model.check_stack_attributes(stacks, event, exclude_indices)
                    
            if stack_index == -1:        
                stack_index = event_activity_model.assign_to_sequence(event, stacks, 4, exclude_indices)
            
            # for elements that are not matchable based on 2 sequences we fall back to stream index
            if stack_index == -1: 
                stack_index = search_stream_index(stacks, event, exclude_indices)    
            
            # fallback - no match add to first stack
            if stack_index == -1:
                res = next((i for i in range(len(stacks)) if i not in exclude_indices and stacks[i].confidence),-1)
                stack_index = res
                
            stacks[stack_index].append_event(event)
            
            
    if activity_action == "Activity End":
        stack_index = search_stack_for_request_frame(event.origin_request_frame, stacks)
        stacks[stack_index].append_event(event)
        
        if event.confidence: 
            if len(stacks) > 1: 
                stack = stacks.pop(stack_index)
                label = activity_type_classifier.classify_stack(stack)
                stack.label = label 
                stacks_out.append(stack)
                
            else: 
                event.confidence = False
                    
    # Loop through all currently open stacks
    for idx, stack in enumerate(stacks):
        last_event = stack[-1]
        # check for non-confident "No Action" Classifications. These could be "Activity End" Instead
        if not last_event.confidence and last_event.activity_action == "NoAction":
            # If a stack has not been continued for N event loops 
            if event_loop_index - last_event.event_loop_index > EVENT_LOOP_CUTOFF_NO_ACTION:
                label = activity_type_classifier.classify_stack(stack)
                stack.label = label 
                stacks.pop(idx)
                stacks_out.append(stack)
               
                
    for idx, stack in enumerate(stacks): 
        last_event = stack.events[-1]
        if not last_event.confidence and last_event.activity_action == "Activity End": 
            if event_loop_index - last_event.event_loop_index > EVENT_LOOP_CUTOFF_END_EVENT: 
                label = activity_type_classifier.classify_stack(stack)
                stack.label = label            
                # we are now sure to pop the stack.
                stacks.pop(idx)
                stacks_out.append(stack)  
            
                
    end_time = time.time()
    processing_times.append(end_time - start_time)
    
for stack in stacks: 
    label = activity_type_classifier.classify_stack(stack)
    stack.label = label      
    stacks_out.append(stack)        

GenerateJobApplicationActivity
ResumeReviewActivity
GenerateJobApplicationActivity
ResumeReviewActivity
GenerateJobApplicationActivity
ResumeReviewActivity
ScheduleAnInterviewActivityCall
GenerateJobApplicationActivity
ResumeReviewActivity
PerformAnInterviewCall
GenerateJobApplicationActivity
ResumeReviewActivity
GenerateJobApplicationActivity
ResumeReviewActivity
ScheduleAnInterviewActivityCall
GenerateJobApplicationActivity
PerformAnInterviewCall
ResumeReviewActivity
ScheduleAnInterviewActivityCall
GenerateJobApplicationActivity
ScheduleAnInterviewMeeting
ResumeReviewActivity
NO MATCH RES 16032
GenerateJobApplicationActivity
PerformAnInterviewCall
PerformAnInterviewMeeting
ScheduleAnInterviewActivityCall
ScheduleAnInterviewMeeting
GenerateJobApplicationActivity
ResumeReviewActivity
PerformAnInterviewCall
PerformAnInterviewMeeting
ScheduleAnInterviewActivityCall
ResumeReviewActivity
ScheduleAnInterviewMeeting
ContractProposal
PerformAnInterviewCall
PerformAnInterviewMeeting


In [125]:
# create Evaluation Data Frame
start = [stack[0].frame_number for stack in stacks_out]
end = [stack[-1].frame_number for stack in stacks_out]
label = [stack.label for stack in stacks_out]
case_id = [stack.case_id["id"] for stack in stacks_out]
res_df = pd.DataFrame({"start_pred":start, "end_pred":end,"label": label, "case_id": case_id})

eval_df = df_gt[["start", "actual_end","activity_name","bp_id"]].merge(res_df,how="left", left_on ="start", right_on = "start_pred").fillna(-1)
eval_df["pred_true"] = eval_df["actual_end"] == eval_df["end_pred"]
eval_df["label_true"] = eval_df["activity_name"] == eval_df["label"]
print(f"Accuracy of matching start and end sequences: {eval_df['pred_true'].mean()}")

bp_id_norm = eval_df["bp_id"] - eval_df["bp_id"].min() +1

eval_df["case_id_true"] = bp_id_norm == eval_df["case_id"]
print(f"Overall matching accuracy: {0.5 + eval_df['pred_true'].mean()/2}")

Accuracy of matching start and end sequences: 1.0
Overall matching accuracy: 1.0


In [128]:
# We evaluate the Activity Type Classification on a per-event Basis
from event_loop.activity_boundaries import extract_labels

test_labels = extract_labels(df_test["ActivityAction"])
# Create a Dataframe with all events and their frame numbers
df_aa_test = pd.DataFrame(df_test[["frame.number", "ActivityAction"]])
# Set type of every event to no action 
df_aa_test["ActivityAction"] = "NoAction"
# Merge Predicted end events by frame number
df_aa_test.loc[df_aa_test["frame.number"].isin(eval_df["end_pred"]), "ActivityAction"] = "Activity End"
# Merge predicted start events by frame number
df_aa_test.loc[df_aa_test["frame.number"].isin(eval_df["start_pred"]), "ActivityAction"] = "Activity Start"
print(classification_report(test_labels, df_aa_test["ActivityAction"]))

                precision    recall  f1-score   support

  Activity End       1.00      1.00      1.00        37
Activity Start       1.00      1.00      1.00        37
      NoAction       1.00      1.00      1.00      1239

      accuracy                           1.00      1313
     macro avg       1.00      1.00      1.00      1313
  weighted avg       1.00      1.00      1.00      1313


In [129]:
print(classification_report(eval_df["activity_name"], eval_df["label"]))

                                 precision    recall  f1-score   support

               ContractProposal       1.00      1.00      1.00         1
 GenerateJobApplicationActivity       1.00      1.00      1.00        10
         PerformAnInterviewCall       1.00      1.00      1.00         5
      PerformAnInterviewMeeting       1.00      1.00      1.00         3
           ResumeReviewActivity       1.00      1.00      1.00        10
ScheduleAnInterviewActivityCall       1.00      1.00      1.00         5
     ScheduleAnInterviewMeeting       1.00      1.00      1.00         3

                       accuracy                           1.00        37
                      macro avg       1.00      1.00      1.00        37
                   weighted avg       1.00      1.00      1.00        37


In [130]:
print(classification_report(eval_df["activity_name"], eval_df["label"]))

                                 precision    recall  f1-score   support

               ContractProposal       1.00      1.00      1.00         1
 GenerateJobApplicationActivity       1.00      1.00      1.00        10
         PerformAnInterviewCall       1.00      1.00      1.00         5
      PerformAnInterviewMeeting       1.00      1.00      1.00         3
           ResumeReviewActivity       1.00      1.00      1.00        10
ScheduleAnInterviewActivityCall       1.00      1.00      1.00         5
     ScheduleAnInterviewMeeting       1.00      1.00      1.00         3

                       accuracy                           1.00        37
                      macro avg       1.00      1.00      1.00        37
                   weighted avg       1.00      1.00      1.00        37
